# สำหรับ “แยกโครงสร้าง PDF

In [11]:
from pypdf import PdfReader

# ตัวทดลองที่ 1: แยก Page → Paragraph (พื้นฐานที่สุด)
def extract_paragraphs(pdf_path: str):
    reader = PdfReader(pdf_path)
    results = []

    for page_idx, page in enumerate(reader.pages):
        text = page.extract_text()
        if not text:
            continue

        paragraphs = [
            p.strip()
            for p in text.split("\n")
            if len(p.strip()) > 5
        ]

        for p in paragraphs:
            results.append({
                "doc_id": pdf_path,
                "page": page_idx + 1,
                "block_type": "paragraph",
                "section": None,
                "text": p
            })

    return results


In [12]:
import re
from pypdf import PdfReader

def extract_with_sections(pdf_path: str):
    reader = PdfReader(pdf_path)
    results = []
    current_section = "Unknown"

    for page_idx, page in enumerate(reader.pages):
        text = page.extract_text()
        if not text:
            continue

        lines = text.split("\n")

        for line in lines:
            clean = line.strip()

            # heuristic: heading ตัวใหญ่ / มีเลขนำ
            if re.match(r"^[0-9]+\.", clean) or clean.isupper():
                current_section = clean
                results.append({
                    "doc_id": pdf_path,
                    "page": page_idx + 1,
                    "block_type": "heading",
                    "section": current_section,
                    "text": clean
                })
                continue

            if len(clean) > 5:
                results.append({
                    "doc_id": pdf_path,
                    "page": page_idx + 1,
                    "block_type": "paragraph",
                    "section": current_section,
                    "text": clean
                })

    return results


In [13]:
import pdfplumber

def extract_tables(pdf_path: str):
    results = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_idx, page in enumerate(pdf.pages):
            tables = page.extract_tables() or []

            for t_idx, table in enumerate(tables):
                flat_rows = []

                for row in table:
                    if not row:
                        continue

                   
                    clean_cells = [
                        str(cell).strip()
                        for cell in row
                        if cell is not None and str(cell).strip() != ""
                    ]

                    if clean_cells:
                        flat_rows.append(" | ".join(clean_cells))

                if flat_rows:
                    results.append({
                        "doc_id": pdf_path,
                        "page": page_idx + 1,
                        "block_type": "table",
                        "section": None,
                        "text": " ; ".join(flat_rows)
                    })

    return results


In [14]:
import pandas as pd

pdf_path = "dsi2566.pdf"

paragraphs = extract_with_sections(pdf_path)
tables = extract_tables(pdf_path)

structured = paragraphs + tables
df_str = pd.DataFrame(structured)

print(df_str.head())


        doc_id  page block_type  section                                  text
0  dsi2566.pdf     1  paragraph  Unknown              หลักสูตรวิทยาศาสตรบัณฑิต
1  dsi2566.pdf     1  paragraph  Unknown  สาขาวิชาวิทยาศาสตร์และนวัตกรรมข้อมูล
2  dsi2566.pdf     1  paragraph  Unknown          (หลักสูตรปรับปรุง พ.ศ. 2566)
3  dsi2566.pdf     1  paragraph  Unknown                   หลักสูตรพหุวิทยาการ
4  dsi2566.pdf     1  paragraph  Unknown            หลักสูตรความร่วมมือระหว่าง


In [15]:
df_str

,doc_id,page,block_type,section,text
0,dsi2566.pdf,1,paragraph,Unknown,หลักสูตรวิทยาศาสตรบัณฑิต
1,dsi2566.pdf,1,paragraph,Unknown,สาขาวิชาวิทยาศาสตร์และนวัตกรรมข้อมูล
2,dsi2566.pdf,1,paragraph,Unknown,(หลักสูตรปรับปรุง พ.ศ. 2566)
3,dsi2566.pdf,1,paragraph,Unknown,หลักสูตรพหุวิทยาการ
4,dsi2566.pdf,1,paragraph,Unknown,หลักสูตรความร่วมมือระหว่าง
...,...,...,...,...,...
2748,dsi2566.pdf,73,table,None,หมวดวิชา | รหัสวิชา | รายวิชา | ผลลัพธ์การเรีย...
2749,dsi2566.pdf,74,table,None,หมวดวิชา | รหัสวิชา | รายวิชา | ผลลัพธ์การเรีย...
2750,dsi2566.pdf,75,table,None,หมวดวิชา | รหัสวิชา | รายวิชา | ผลลัพธ์การเรีย...
2751,dsi2566.pdf,76,table,None,หมวดวิชา | รหัสวิชา | รายวิชา | ผลลัพธ์การเรีย...


In [50]:
# บันทึกเป็น CSV
df.to_csv("dsi2566_structured.csv", index=False, encoding="utf-8-sig")

# ทดลอง DSPy + LLM

In [ ]:
# ============================================================
# Curriculum STRUCTURE-AWARE QA Agent
# DSPy + Ollama (FINAL + PROGRESS BAR)
# ============================================================

import json
import requests
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import dspy

# ============================================================
# 1) CONFIG
# ============================================================

EMBEDDING_API = "http://localhost:11434/api/embeddings"
EMBED_MODEL = "bge-m3:567m"

LLM_URL = "http://localhost:11434"
MODEL_NAME = "gpt-oss:20b"

TOP_K = 6

# ============================================================
# 2) EMBEDDING (WITH PROGRESS BAR)
# ============================================================

def embed_texts(texts: list[str]) -> np.ndarray:
    vectors = []

    for t in tqdm(
        texts,
        desc="Creating embeddings",
        unit="chunk"
    ):
        r = requests.post(
            EMBEDDING_API,
            json={
                "model": EMBED_MODEL,
                "prompt": t
            },
            timeout=120
        )
        r.raise_for_status()
        vectors.append(r.json()["embedding"])

    return np.array(vectors)

# ============================================================
# 3) BUILD CORPUS FROM DF (WITH PROGRESS BAR)
# ============================================================

def build_corpus(df: pd.DataFrame):
    texts, meta = [], []

    for _, r in tqdm(
        df.iterrows(),
        total=len(df),
        desc="Building retrieval corpus",
        unit="block"
    ):
        blob = f"""
[SECTION] {r['section']}
[TYPE] {r['block_type']}
[TEXT]
{r['text']}
""".strip()

        texts.append(blob)
        meta.append({
            "section": r["section"],
            "page": r["page"],
            "block_type": r["block_type"]
        })

    return texts, meta

# ============================================================
# 4) RETRIEVER
# ============================================================

class Retriever:
    def __init__(self, texts, meta, embeddings):
        self.texts = texts
        self.meta = meta
        self.embeddings = embeddings

    def search(self, query: str, k: int = TOP_K):
        q_emb = embed_texts([query])
        sims = cosine_similarity(q_emb, self.embeddings)[0]
        idx = sims.argsort()[-k:][::-1]

        return [
            {
                "text": self.texts[i],
                "meta": self.meta[i],
                "score": float(sims[i])
            }
            for i in idx
        ]

# ============================================================
# 5) PASS 1 — STRUCTURE EXTRACTION AGENT
# ============================================================

class CurriculumStructureSignature(dspy.Signature):
    """
    Extract curriculum structure as JSON.
    JSON ONLY. No explanation.
    """
    context = dspy.InputField()
    structure = dspy.OutputField()


class CurriculumStructureAgent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(CurriculumStructureSignature)

    def forward(self, context: str):
        return self.predict(context=context)

# ============================================================
# 6) BUILD GLOBAL CONTEXT (WITH PROGRESS BAR)
# ============================================================

def build_global_context(df: pd.DataFrame, max_chars=12000):
    chunks = []

    for _, r in tqdm(
        df.iterrows(),
        total=len(df),
        desc="Building global curriculum context",
        unit="block"
    ):
        chunks.append(
            f"[{r['block_type']} | {r['section']}]\n{r['text']}"
        )

    return "\n\n".join(chunks)[:max_chars]

# ============================================================
# 7) PASS 2 — QA SIGNATURE
# ============================================================

class CurriculumQASignature(dspy.Signature):
    """
   คุณคือ “บรรณารักษ์ดิจิทัล (Digital Librarian Agent)”
หน้าที่ของคุณคือ:
- ค้นหา
- คัดกรอง
- อ้างอิง
- และสรุปข้อมูล
จากเอกสารที่ให้มาอย่างเป็นระบบและตรวจสอบได้

กฎการตอบ:
1. ตอบคำถามโดยใช้ข้อมูลจาก CONTEXT ที่ให้มาเท่านั้น
2. ห้ามใช้ความรู้ภายนอก หรือคาดเดาเพิ่มเติม
3. หากไม่พบข้อมูลที่เกี่ยวข้องในเอกสาร ให้ตอบว่า:
   "ไม่พบข้อมูลในเอกสาร"
4. ตอบเป็นภาษาไทย
5. จัดคำตอบอย่างเป็นระบบ ชัดเจน และเป็นกลางในเชิงวิชาการ
6. ใช้ภาษาที่สุภาพ แม่นยำ และมีลักษณะเชิงสารสนเทศแบบบรรณารักษ์

รูปแบบคำตอบที่แนะนำ:
- หัวข้อ / ประเด็นหลัก
- รายละเอียดจากเอกสาร
- (ถ้ามี) การจัดกลุ่มหรือเรียงลำดับตามโครงสร้างเอกสาร
    """
    question = dspy.InputField()
    context = dspy.InputField()
    answer = dspy.OutputField()

# ============================================================
# 8) STRUCTURE-AWARE QA AGENT (ReAct)
# ============================================================

class CurriculumQAAgent(dspy.Module):
    def __init__(self, retriever, curriculum_structure: str):
        super().__init__()
        self.retriever = retriever
        self.curriculum_structure = curriculum_structure

        self.react = dspy.ReAct(
            CurriculumQASignature,
            tools=[]
        )

    def forward(self, question: str):
        docs = self.retriever.search(question)

        retrieved = "\n\n".join(
            f"[{d['meta']['section']} | page {d['meta']['page']}]\n{d['text']}"
            for d in docs
        )

        context = f"""
### CURRICULUM STRUCTURE (GROUND TRUTH)
{self.curriculum_structure}

### RELEVANT DOCUMENTS
{retrieved}
""".strip()

        return self.react(
            question=question,
            context=context
        )

# ============================================================
# 9) MAIN PIPELINE (WITH PROGRESS BAR)
# ============================================================

def main(df: pd.DataFrame):
    # -----------------------------
    # Init LLM
    # -----------------------------
    print("Initializing LLM...")
    lm = dspy.LM(
        f"ollama/{MODEL_NAME}",
        api_base=LLM_URL,
        cache=False
    )
    dspy.configure(lm=lm)

    # -----------------------------
    # PASS 1: Extract Structure
    # -----------------------------
    print("\nExtracting curriculum structure...")
    global_context = build_global_context(df)

    structure_agent = CurriculumStructureAgent()
    with tqdm(total=1, desc="Curriculum structure reasoning") as pbar:
        structure_result = structure_agent(context=global_context)
        pbar.update(1)

    curriculum_structure = structure_result.structure

    print("\nCURRICULUM STRUCTURE (JSON):")
    print(curriculum_structure)

    # -----------------------------
    # Build Retriever
    # -----------------------------
    print("\nBuilding retrieval index...")
    corpus_texts, corpus_meta = build_corpus(df)
    embeddings = embed_texts(corpus_texts)

    retriever = Retriever(
        corpus_texts,
        corpus_meta,
        embeddings
    )

    # -----------------------------
    # PASS 2: QA Agent
    # -----------------------------
    agent = CurriculumQAAgent(
        retriever,
        curriculum_structure
    )

    print("\nStructure-aware Curriculum QA Agent พร้อมใช้งาน")
    print("พิมพ์คำถามเกี่ยวกับหลักสูตร (exit เพื่อออก)\n")

    while True:
        q = input("คำถาม: ").strip()
        if q.lower() in ["exit", "quit"]:
            break

        result = agent(question=q)
        print("\nคำตอบ:")
        print(result.answer)
        print("-" * 60)

# ============================================================
# 10) ENTRY
# ============================================================

if __name__ == "__main__":
    """
    df ต้องมี column:
    - text
    - section
    - page
    - block_type
    """

    df = df_str   # ← ใส่ DataFrame ของคุณตรงนี้
    main(df)


Initializing LLM...

Extracting curriculum structure...


Curriculum structure reasoning: 100%|████████████████████████████████████████████████████| 1/1 [00:58<00:00, 58.69s/it]



CURRICULUM STRUCTURE (JSON):
{
  "title": "หลักสูตรวิทยาศาสตรบัณฑิต",
  "institution": "มหาวิทยาลัยธรรมศาสตร์",
  "faculty": "วิทยาลัยสหวิทยาการ",
  "sections": {
    "1": {
      "title": "ข้อมูลทั่วไป",
      "subsections": {
        "1.1": {
          "title": "รหัสและชื่อหลักสูตร",
          "content": [
            {
              "รหัสหลักสูตร": "20182067117526"
            },
            {
              "ชื่อหลักสูตร (ภาษาไทย)": "หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์และนวัตกรรมข้อมูล"
            },
            {
              "ชื่อหลักสูตร (ภาษาอังกฤษ)": "Bachelor of Science Program in Data Science and Innovation"
            }
          ]
        },
        "1.2": {
          "title": "ชื่อปริญญาและสาขาวิชา",
          "content": [
            {
              "ชื่อเต็ม (ภาษาไทย)": "วิทยาศาสตรบัณฑิต (วิทยาศาสตร์และนวัตกรรมข้อมูล)"
            },
            {
              "ชื่อย่อ (ภาษาไทย)": "วท.บ. (วิทยาศาสตร์และนวัตกรรมข้อมูล)"
            },
            {
         

Creating embeddings: 100%|██████████████████████████████████████████████████████| 2753/2753 [16:52<00:00,  2.72chunk/s]


Structure-aware Curriculum QA Agent พร้อมใช้งาน
พิมพ์คำถามเกี่ยวกับหลักสูตร (exit เพื่อออก)



...

Creating embeddings: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.65chunk/s]



คำตอบ:
**ชื่อหลักสูตร**  
หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์และนวัตกรรมข้อมูล  

**หน่วยงานที่เปิดสอน**  
มหาวิทยาลัยธรรมศาสตร์ – วิทยาลัยสหวิทยาการ
------------------------------------------------------------


คำถาม:  ปี 1 เรียนอะไรบ้าง


Creating embeddings: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.40chunk/s]



คำตอบ:
**ปี 1 – รายวิชาที่เรียน**

| ภาคเรียน | รหัส | ชื่อวิชา | หน่วยกิต |
|-----------|------|----------|----------|
| **ภาคเรียนที่ 1** | มธ.100 | พลเมืองกับการลงมือแก้ปัญหา | 3 |
| | มธ.106 | ความคิดสร้างสรรค์และการสื่อสาร | 3 |
| | มธ.107 | ทักษะดิจิทัลกับการแก้ปัญหา | 3 |
| | มธ.108 | การพัฒนาและจัดการตนเอง | 3 |
| | สก.200 | วิทยาศาสตร์ข้อมูลสำหรับชีวิตประจำวัน | 3 |
| | วสห.104 | การเขียนโปรแกรมเพื่อวิเคราะห์ข้อมูล | 3 |
| **ภาคเรียนที่ 2** | สษ.105 | ทักษะการสื่อสารทางภาษาอังกฤษ | 3 |
| | ศศ.101 | การคิด อ่านและเขียนอย่างมีวิจารณญาณ | 3 |
| | มธ.155 | สถิติพื้นฐาน | 3 |
| | สก.202 | นวัตกรรมเพื่อสังคมและผู้ประกอบการเพื่อสังคม | 3 |
| | วข.201 | การจัดการฐานข้อมูล | 3 |
| | วข.208 | โครงสร้างข้อมูลและขั้นตอนวิธีสำหรับการวิเคราะห์ข้อมูล | 3 |

**สรุปผลการเรียนรู้ที่คาดหวังในปี 1**  
- ความรู้ด้านคณิตศาสตร์ (พีชคณิตเชิงเส้นและความน่าจะเป็น)  
- ความเข้าใจเกี่ยวกับคอมพิวเตอร์, ฐานข้อมูล และการพัฒนาซอฟต์แวร์  
- ทักษะการเขียนโปรแกรมภาษา Python ในการแก้ปัญหา  
- ทักษะการสื่อสารด้ว